In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from sklearn.externals import joblib

In [8]:
data_train = pd.read_csv("C:/Users/Weng/Desktop/Web economics/Dataset/train.csv")
df_v = pd.read_csv("C:/Users/Weng/Desktop/Web economics/Dataset/validation.csv")
#df_test = pd.read_csv("C:/Users/Weng/Desktop/Web economics/Dataset/test.csv")

In [9]:
#df_predict = pd.read_csv("C:/Users/Weng/Desktop/Web economics/Github/Web-Economics-master/click_pred.csv")

In [10]:
#df_bidprice = pd.read_csv("C:/Users/Weng/Desktop/Web economics/Github/Web-Economics-master/out.csv")

In [11]:
#df_v[df_v.bidid == 'd939cd3025fba6ee162860e5cb4fad21']

In [12]:
#df_v.ix[df_predict[df_predict[' click'] == 1].index].click

In [13]:
#df_predict_acc = df_predict

In [14]:
#df_predict_acc['v_clicks'] = df_v.loc[df_predict[df_predict[' click'] == 1].index, 'click']  # Locator of the index

In [15]:
#print ('Click prediction accuracy:  '+ str(round(((df_predict_acc['v_clicks'].sum() / df_predict_acc[' click'].sum())*100),3)))

#### Bidding model

In [16]:
#Take a quarter of the trainning set as a sample 
data_train = data_train.sample(frac = 1,random_state = 20).reset_index(drop=True)

In [17]:
def CTR_calc(data):
    Click_t_rate= "Click Through Rate= "+ str(round((np.count_nonzero(data.click.values)/data.shape[0])*100,4)) + ' %'
    return (Click_t_rate)

In [18]:
data_train.shape

(2697738, 26)

In [19]:
CTR_calc(data_train)

'Click Through Rate= 0.0754 %'

In [20]:
#Dropping un-interesting columns
data_train= data_train.drop(['bidid','userid','logtype','url','urlid','keypage','bidprice','payprice'],1)

In [21]:
data_train.columns

Index(['click', 'weekday', 'hour', 'useragent', 'IP', 'region', 'city',
       'adexchange', 'domain', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'advertiser',
       'usertag'],
      dtype='object')

In [22]:
#Second drop- to improve performance 
#data_train = data_train.drop(['IP','domain','usertag'],1)

In [23]:
data_train.head()

,click,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,advertiser,usertag
0,0,5,10,windows_ie,119.249.43.*,3,9,3,5F1RQS9rg5scFsf,News_F_Width1,1000,90,0,0,80,832b91d59d0cb5731431653204a76c0e,1458,"13042,10076,10031,10057,10075,10006,13403,1006..."
1,0,1,16,windows_ie,110.241.12.*,3,4,3,3c47edf60cc139641b6395e123a6e155,ALLINONE_F_Width1,1000,90,Na,Na,70,10734,2821,"14273,10102,10006,10110,10031,13403,10063"
2,0,0,23,windows_ie,61.146.117.*,216,230,3,7ed515fe566938ee6cfbb6ebb7ea4995,Sports_F_Upright,300,250,Na,Na,10,10722,2821,"10059,10684,10083,10024,10006,16706,10111,1005..."
3,0,1,18,windows_ie,112.15.2.*,94,104,3,3043163ba84753b4b51dd3290caeae67,tips_F_Width2,1000,90,Na,Na,50,10734,2821,null
4,0,1,3,windows_ie,223.241.165.*,106,114,3,3SCYZrn0Qo18XMB4JKTI,discuz_18316225_005,728,90,0,0,20,b90c12ed2bd7950c6027bf9c6937c48a,3386,null


### Preprocessing of data

In [24]:
#Removing strings in useragent
data_train.update(data_train['useragent'].str.replace('_ie', ''))
data_train.update(data_train['useragent'].str.replace('_firefox', ''))
data_train.update(data_train['useragent'].str.replace('_safari', ''))
data_train.update(data_train['useragent'].str.replace('_chrome', ''))
data_train.update(data_train['useragent'].str.replace('_other', ''))
data_train.update(data_train['useragent'].str.replace('_opera', ''))
data_train.update(data_train['useragent'].str.replace('_maxthon', ''))
data_train.update(data_train['useragent'].str.replace('_theworld', ''))
data_train.update(data_train['useragent'].str.replace('_sogou', ''))

#### Encoding the data

In [25]:
from sklearn.preprocessing import LabelEncoder

In [26]:
le=LabelEncoder()

In [27]:
le_ad= LabelEncoder()

In [28]:
le_ad.fit(['1458', '2821', '3386', '2261', '3476', '2259', '3358', '2997', '3427'])

LabelEncoder()

In [29]:
data_train['advertiser'] = le_ad.transform(data_train.advertiser)

In [30]:
le_user= LabelEncoder()

In [31]:
data_train['useragent']= le_user.fit_transform(data_train.useragent)

In [32]:
le_adexchange= LabelEncoder()

In [33]:
le_adexchange.fit(['3', '2', '1', 'null', '4'])

LabelEncoder()

In [34]:
data_train['slotprice'] = le.fit_transform(data_train.slotprice)

In [35]:
data_train['adexchange']= le_user.fit_transform(data_train.adexchange)

In [36]:
data_train['slotwidth']=le_user.fit_transform(data_train.slotwidth)

In [37]:
data_train['slotheight']=le_user.fit_transform(data_train.slotheight)

In [38]:
data_train['slotvisibility']=le_user.fit_transform(data_train.slotvisibility)

In [39]:
data_train['slotid']=le_user.fit_transform(data_train.slotformat)

In [40]:
data_train['slotformat']=le_user.fit_transform(data_train.slotformat)

In [41]:
data_train['city']=le_user.fit_transform(data_train.city)

In [42]:
data_train['region']=le_user.fit_transform(data_train.region)

In [43]:
data_train['creative']=le_user.fit_transform(data_train.creative)

#### delete if necessary

In [44]:
data_train['usertag']= le_user.fit_transform(data_train.usertag)

In [45]:
data_train['domain']= le_user.fit_transform(data_train.domain)

In [46]:
data_train['IP']= le_user.fit_transform(data_train.IP)

In [47]:
#third drop- to improve performance 
#data_train = data_train.drop(['slotwidth','slotheight','slotvisibility','slotformat'],1)

In [48]:
data_train.city.unique()

array([  9,   4, 224, 103, 113,  13,  75, 262, 216, 106,   1, 314, 107,
       211, 228, 124, 250, 196, 255, 105, 170,  47,   2,  97,   0, 213,
       215, 349,  78, 283, 252,  84, 202, 270, 147, 148, 198, 248, 134,
       221,  64, 310,  89, 182,  96, 350, 217, 225,  14,  55, 145, 233,
       242, 179, 153, 164, 125, 166,  12, 151, 319, 128,   7, 230, 220,
       143,  74,  30, 214,  90, 227, 281, 243, 315, 308, 136, 174,  32,
        85, 146,  98,  94, 183, 234, 155, 152,  81,  80, 363,  29, 222,
       236, 154, 360, 115, 239, 208, 311, 126,  41,  82, 264, 132,  79,
        99, 149, 210, 186, 197,   5,  83, 190,  17, 192, 200, 100,  11,
       207, 127,  88, 165, 284, 235, 282, 130, 273, 119, 185,  42, 118,
       229, 150, 176,  18, 356,  95,  52,  20, 104,  26, 307,  23, 274,
       257, 168, 256, 110, 312, 159, 265, 121, 171,   6,  92, 244, 251,
       258,  49, 204, 316, 109, 156, 218, 269,  58,  76, 291, 334, 187,
       268,  34, 101,  19, 219, 322, 237,  62,  35, 226,  45,  4

In [49]:
data_train

,click,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,advertiser,usertag
0,0,5,10,5,176985,3,9,2,4863,0,20,3,0,0,77,96,0,667707
1,0,1,16,5,35240,3,4,2,2919,3,20,3,7,3,67,12,3,802511
2,0,0,23,5,500865,19,224,2,7525,3,7,10,7,3,7,9,3,368821
3,0,1,18,5,62962,11,103,2,1446,3,20,3,7,3,47,12,3,814363
4,0,1,3,5,424549,12,113,2,2720,0,14,3,0,0,17,104,6,814363
5,0,4,17,5,484122,3,13,2,7525,3,7,10,7,3,47,25,2,256965
6,0,4,23,5,92554,8,75,1,20250,0,2,13,0,0,145,52,6,395926
7,0,4,11,5,236266,23,262,1,17892,0,7,10,1,0,2,120,8,316657
8,0,0,11,5,500488,19,216,1,6622,3,14,3,8,3,2,83,1,192199
9,0,5,13,5,251184,12,106,2,4863,0,20,3,0,0,77,97,6,716610


In [50]:
data_train.columns

Index(['click', 'weekday', 'hour', 'useragent', 'IP', 'region', 'city',
       'adexchange', 'domain', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'advertiser',
       'usertag'],
      dtype='object')

In [51]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
data_train = pd.DataFrame(mm.fit_transform(data_train))
data_train.columns = ['click', 'weekday', 'hour', 'useragent', 'IP', 'region', 'city',
       'adexchange', 'domain', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'advertiser',
       'usertag']

In [52]:
#One hot encoding - arrange in order - only run once then restart because it crashes - not necessary for random forests
#data_train = pd.get_dummies(data_train, columns = ['weekday','advertiser','hour','adexchange','slotformat','slotvisibility','useragent'])# columns = ['weekday','advertiser','hour','slotheight','slotwidth']

In [53]:
#Features and gold labels
#x_feat= data_train.ix[:,1:] # might have different columns when one hot another sample
#y_targets = data_train['click']

#### Validation pre-processing

In [54]:
#Drop columns
df_v = df_v.drop(['bidid','userid','logtype','url','urlid','keypage','bidprice','payprice'],1)

In [55]:
#Removing strings in useragent
df_v.update(df_v['useragent'].str.replace('_ie', ''))
df_v.update(df_v['useragent'].str.replace('_firefox', ''))
df_v.update(df_v['useragent'].str.replace('_safari', ''))
df_v.update(df_v['useragent'].str.replace('_chrome', ''))
df_v.update(df_v['useragent'].str.replace('_other', ''))
df_v.update(df_v['useragent'].str.replace('_opera', ''))
df_v.update(df_v['useragent'].str.replace('_maxthon', ''))
df_v.update(df_v['useragent'].str.replace('_theworld', ''))
df_v.update(df_v['useragent'].str.replace('_sogou', ''))

In [56]:
#Encoder transform for the validation set
df_v['advertiser'] = le.fit_transform(df_v.advertiser)
df_v['useragent']= le_user.fit_transform(df_v.useragent)
df_v['slotprice'] = le.fit_transform(df_v.slotprice)
df_v['adexchange']= le_user.fit_transform(df_v.adexchange)
df_v['slotwidth']=le_user.fit_transform(df_v.slotwidth)
df_v['slotheight']=le_user.fit_transform(df_v.slotheight)
df_v['slotvisibility']=le_user.fit_transform(df_v.slotvisibility)
df_v['slotid']=le_user.fit_transform(df_v.slotformat)
df_v['slotformat']=le_user.fit_transform(df_v.slotformat)
df_v['city']=le_user.fit_transform(df_v.city)
df_v['region']=le_user.fit_transform(df_v.region)
df_v['creative']=le_user.fit_transform(df_v.creative)
#Additional
df_v['usertag']= le_user.fit_transform(df_v.usertag)
df_v['domain']= le_user.fit_transform(df_v.domain)
df_v['IP']= le_user.fit_transform(df_v.IP)


In [57]:
df_v.shape

(299749, 18)

In [58]:
df_v.head()

,click,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,advertiser,usertag
0,0,5,14,5,103240,19,226,0,6592,1,7,10,0,1,0,51,7,5214
1,0,3,14,0,49519,19,211,4,6315,3,8,0,5,3,115,13,4,114127
2,0,1,13,5,186114,15,148,2,810,0,20,3,0,0,67,112,7,114127
3,0,5,18,5,166083,15,147,2,3159,0,20,3,0,0,67,106,7,114127
4,0,5,14,5,129221,22,250,1,6315,0,9,11,2,0,2,128,6,114127


In [59]:
df_v.columns

Index(['click', 'weekday', 'hour', 'useragent', 'IP', 'region', 'city',
       'adexchange', 'domain', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'advertiser',
       'usertag'],
      dtype='object')

In [60]:
df_v = pd.DataFrame(mm.fit_transform(df_v))
df_v.columns = ['click', 'weekday', 'hour', 'useragent', 'IP', 'region', 'city',
       'adexchange', 'domain', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'advertiser',
       'usertag']

In [61]:
#Separate the training set 

X_train = data_train.ix[:,1:]
y_train = data_train['click']
#Need to repeat encoding for the validation set + validation test set
X_test = df_v.ix[:,1:]
y_test = df_v['click']

In [62]:
y_train.shape

(2697738,)

In [63]:
#Work with x_train-Bid request and y_train - clicks
X_train.head()

,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,advertiser,usertag
0,0.833333,0.434783,1.0,0.343308,0.088235,0.024390,0.5,0.201902,0.0,1.00,0.230769,0.0,0.0,0.272085,0.744186,0.000,0.819913
1,0.166667,0.695652,1.0,0.068357,0.088235,0.010840,0.5,0.121191,1.0,1.00,0.230769,0.7,1.0,0.236749,0.093023,0.375,0.985446
2,0.000000,1.000000,1.0,0.971555,0.558824,0.607046,0.5,0.312422,1.0,0.35,0.769231,0.7,1.0,0.024735,0.069767,0.375,0.452895
3,0.166667,0.782609,1.0,0.122131,0.323529,0.279133,0.5,0.060035,1.0,1.00,0.230769,0.7,1.0,0.166078,0.093023,0.375,1.000000
4,0.166667,0.130435,1.0,0.823521,0.352941,0.306233,0.5,0.112929,0.0,0.70,0.230769,0.0,0.0,0.060071,0.806202,0.750,1.000000


#### PCTR prediction feature engineering 

In [59]:
#Using random forest classifier to predict clicks
from sklearn.ensemble import RandomForestClassifier

#### Sample split is important

#Best random forest so far  #predicted 149 clicks with 18 accurate clicks [around 30mins]
clf = RandomForestClassifier(n_estimators=100, max_features = 5, oob_score = True, random_state = 2, 
                             min_impurity_split= 0.02, min_samples_leaf = 10,                                                         
                             min_weight_fraction_leaf= 0,min_samples_split = 10,class_weight= 'balanced_subsample')
                             
```RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features=5,
            max_leaf_nodes=None, min_impurity_split=0.02,
            min_samples_leaf=12, min_samples_split=10,
            min_weight_fraction_leaf=0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=5, verbose=0, warm_start=False) 
```

In [61]:
#Feel free to change the parameters
clf = RandomForestClassifier(n_estimators=1000, max_features = 8, oob_score = True, random_state = 8, 
                             min_impurity_split= 0.04, min_samples_leaf = 12, 
                             min_weight_fraction_leaf= 0,min_samples_split = 8,class_weight= 'balanced_subsample') #balanced_subsample

In [62]:
%%time
clf.fit(X_train, y_train)

Wall time: 7h 52min 21s


RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features=8,
            max_leaf_nodes=None, min_impurity_split=0.04,
            min_samples_leaf=12, min_samples_split=8,
            min_weight_fraction_leaf=0, n_estimators=1000, n_jobs=1,
            oob_score=True, random_state=8, verbose=0, warm_start=False)

In [5]:
#load random forest
clf= joblib.load('random_forest_1.pkl')

In [65]:
clf.predict(X_test)

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [38]:
#dont run this after loading
clf_predict = clf.predict(X_test)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [6]:
clf.oob_score_

0.99702973379920512

In [6]:
for feature, importance in zip(X_train.columns,clf.feature_importances_):
    print (feature, ":", importance)

NameError: name 'X_train' is not defined

In [68]:
acc_df = pd.DataFrame([clf.predict(X_test).transpose(),y_test], index=['Predict','Test']).transpose()

In [72]:
%store acc_df

Stored 'acc_df' (DataFrame)


In [69]:
#Number of clicks 1 = actually predicted 
np.count_nonzero(clf_predict)

NameError: name 'clf_predict' is not defined

In [70]:
df_v.loc[[4932]]

,click,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,advertiser,usertag
4932,1.0,0.333333,0.782609,0.0,0.673227,0.323529,0.260163,1.0,0.692358,1.0,0.4,0.0,0.5,1.0,0.011321,0.101562,0.5,1.0


In [71]:
print('Number of accurate clicks: '+ str(len(acc_df[(acc_df.Predict==1)&(acc_df.Test==1)])))
print('Number of clicks: '+ str(len(acc_df[acc_df.Predict==1]))+ ' out of ' + str(len(acc_df[acc_df.Test==1])))
acc_df[(acc_df.Predict==1)&(acc_df.Test==1)]

Number of accurate clicks: 31
Number of clicks: 555 out of 226


,Predict,Test
2417,1.0,1.0
17526,1.0,1.0
41277,1.0,1.0
45304,1.0,1.0
52906,1.0,1.0
54793,1.0,1.0
55489,1.0,1.0
65866,1.0,1.0
77341,1.0,1.0
77710,1.0,1.0


In [71]:
acc_df[acc_df.Test == 1]

,Predict,Test
478,0.0,1.0
1604,0.0,1.0
2417,1.0,1.0
2746,0.0,1.0
4087,0.0,1.0
4932,0.0,1.0
5910,0.0,1.0
6875,0.0,1.0
8257,0.0,1.0
8572,0.0,1.0


In [71]:
np.count_nonzero(y_train)

2034

#### Feature importance table 

In [16]:
df_feat_imp = pd.Series(data={'weekday' : 0.0480404989746,
'hour' : 0.0811268786201,
'useragent' : 0.0663829361584,
'IP' : 0.139412881838,
'region' : 0.0450016775641,
'city' : 0.101127068119,
'adexchange' : 0.0157262061283,
'domain' : 0.0999075939707,
'slotid' : 0.0074363306999,
'slotwidth' : 0.0273291819777,
'slotheight' : 0.0215029727065,
'slotvisibility' : 0.0292156302147,
'slotformat' : 0.00775163393006,
'slotprice' : 0.0428730374553,
'creative' : 0.0527915814087,
'advertiser' : 0.0309960712712,
'usertag' : 0.183377818963}, name='Importance [%]')

In [17]:
df_feat_imp.index.name = 'Features'

In [19]:
df_feat_imp

Features
IP                0.139413
adexchange        0.015726
advertiser        0.030996
city              0.101127
creative          0.052792
domain            0.099908
hour              0.081127
region            0.045002
slotformat        0.007752
slotheight        0.021503
slotid            0.007436
slotprice         0.042873
slotvisibility    0.029216
slotwidth         0.027329
useragent         0.066383
usertag           0.183378
weekday           0.048040
Name: Importance, dtype: float64

In [20]:
df_feat_imp= df_feat_imp.reset_index()

In [28]:
df_feat_imp.update(round(df_feat_imp.Importance, 5))

In [40]:
df_feat_imp.Importance.sort_values(ascending=False)

15    0.18338
0     0.13941
3     0.10113
5     0.09991
6     0.08113
14    0.06638
4     0.05279
16    0.04804
7     0.04500
11    0.04287
2     0.03100
12    0.02922
13    0.02733
9     0.02150
1     0.01573
8     0.00775
10    0.00744
Name: Importance, dtype: float64

In [49]:
df_feat_imp = df_feat_imp.sort_values('Importance',ascending =False).reset_index(drop = True)

In [70]:
df_feat_imp1 = df_feat_imp[0:9]

In [101]:
df_feat_imp2 = pd.DataFrame(df_feat_imp[9:].index.tolist())

In [106]:
df_feat_imp2 = df_feat_imp[9:].reset_index()

In [100]:
df_feat_imp2

,Features,Importance
9,slotprice,0.04287
10,advertiser,0.03100
11,slotvisibility,0.02922
12,slotwidth,0.02733
13,slotheight,0.02150
14,adexchange,0.01573
15,slotformat,0.00775
16,slotid,0.00744


In [83]:
df_feat_imp1.cdf_feat_imp2)

AttributeError: 'DataFrame' object has no attribute 'concat'

In [110]:
print(pd.concat([df_feat_imp1,df_feat_imp2],axis =1).to_latex())

\begin{tabular}{llrrlr}
\toprule
{} &   Features &  Importance &  index &        Features &  Importance \\
\midrule
0 &    usertag &     0.18338 &    9.0 &       slotprice &     0.04287 \\
1 &         IP &     0.13941 &   10.0 &      advertiser &     0.03100 \\
2 &       city &     0.10113 &   11.0 &  slotvisibility &     0.02922 \\
3 &     domain &     0.09991 &   12.0 &       slotwidth &     0.02733 \\
4 &       hour &     0.08113 &   13.0 &      slotheight &     0.02150 \\
5 &  useragent &     0.06638 &   14.0 &      adexchange &     0.01573 \\
6 &   creative &     0.05279 &   15.0 &      slotformat &     0.00775 \\
7 &    weekday &     0.04804 &   16.0 &          slotid &     0.00744 \\
8 &     region &     0.04500 &    NaN &             NaN &         NaN \\
\bottomrule
\end{tabular}



#### Cross validation

In [7]:
from sklearn.model_selection import cross_val_score

In [8]:
scores = cross_val_score(clf, data_train.ix[:,1:], data_train['click'], cv=5)

NameError: name 'data_train' is not defined